In [ ]:
Всем привет! Кидаю первую домашку. Срок выполнения - 2 недели (до 11 апреля)\
    . Есть 2 варианта задачи, можете выбрать любой, который вам больше нравится, ил же решить оба.

1 вариант: Определение спама (data.zip).\
    У вас есть 2 папки: spam и notspam, содержащие спам и обычные письма, соответственно. Нужно на основании этих данных понять, какие письма из папки unknown являются спамом.\
    Корректным результатом будет файл с расширением .csv формата $"{имя файла};{1, если спам, 0, иначе}", а также список из 30 признаков, которые наиболее сильно помогают отличать спам от не спама

2 вариант: Предсказание принадлежности к партии (Democracy.zip) Описание полей в statTask.txt. 
Для многих данных поле "pidAggr" (с какой партией себя ассоциирует опрашиваемый) неизвестно. Нужно это поле восстановить. При обучении и проверке можно учитывать только данные по республиканцам и демократам. При финальной проверке результатов ответы для людей из других движений учитываться не будут. Корректным результатом будет файл формата: $"{id};{pidAggr}" для каждого неизвестного опрашиваемого, а также список из 5 вопросов, которые нужно задать, чтобы как можно точнее определить принадлежность опрашиваемого.

Для сдачи любой из задач нужно мне на почту khr@skbkontur.ru прислать файл с решением. 

Решение нужно получить при помощи наивного байесовского метода. Но крайне рекомендуется сравнить результат с другими классификаторами, такие как Random Forest, SVM, XGBoost.

Финальный балл будет выставляться в соответствии с распределением точностей решений, где точность - количество угаданных классов / количество всех примеров

In [1]:
import os
from io import open
from typing import Iterator, Tuple


def load_files(path: str) -> Iterator[str]:
    for file in os.listdir(path):
        with open(os.path.join(path, file)) as f:
            yield f.read()

In [2]:
import pandas as pd
from itertools import chain

spam_it = iter((l, 1) for l in load_files('data/spam'))
not_spam_it = iter((l, 0) for l in load_files('data/notSpam'))
df = pd.DataFrame.from_records(chain(spam_it, not_spam_it), columns=('text', 'label'))
df.head()

D:\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


,text,label
0,From blissptht65@yahoo.com Thu Jul 12 06:33:5...,1
1,From corn422@emailisfun.com Tue Jun 26 04:35:...,1
2,From olheie31@usa.net Mon Aug 6 13:04:09 200...,1
3,From lowrate@softtip.net Mon Jun 25 13:28:06 ...,1
4,From wjl74@usa.net Tue Jun 26 08:45:13 2001\n...,1


In [3]:
from textblob import TextBlob


def feature_extraction(text: str):
    lines = text.split('\n')
    text = ' '.join(lines)
    return {
        'character_length': len(text),
        'lines_length': len(lines),
        'word_length': len(TextBlob(text).words)
    }

print(feature_extraction(df.text[0]))

{'word_length': 803, 'lines_length': 154, 'character_length': 5595}


In [4]:
features = df.join(df['text'].map(feature_extraction).apply(pd.Series))
features.head()

,text,label,character_length,lines_length,word_length
0,From blissptht65@yahoo.com Thu Jul 12 06:33:5...,1,5595,154,803
1,From corn422@emailisfun.com Tue Jun 26 04:35:...,1,8558,146,1276
2,From olheie31@usa.net Mon Aug 6 13:04:09 200...,1,21229,856,2478
3,From lowrate@softtip.net Mon Jun 25 13:28:06 ...,1,6355,158,753
4,From wjl74@usa.net Tue Jun 26 08:45:13 2001\n...,1,1388,74,170


In [5]:
features.groupby('label').describe()

<div>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>character_length</th>
      <th>lines_length</th>
      <th>word_length</th>
    </tr>
    <tr>
      <th>label</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="8" valign="top">0</th>
      <th>count</th>
      <td>400.000000</td>
      <td>400.000000</td>
      <td>400.000000</td>
    </tr>
    <tr>
      <th>mean</th>
      <td>3862.590000</td>
      <td>90.060000</td>
      <td>461.017500</td>
    </tr>
    <tr>
      <th>std</th>
      <td>2059.112989</td>
      <td>45.190673</td>
      <td>311.907660</td>
    </tr>
    <tr>
      <th>min</th>
      <td>905.000000</td>
      <td>21.000000</td>
      <td>100.000000</td>
    </tr>
    <tr>
      <th>25%</th>
      <td>2884.000000</td>
      <td>69.000000</td>
      <td>343.000000</td>
    </tr>
    <tr>
      <th>50%</th>
      <td>3417.000000</td>
      <td>81.000000</td>
      <td>397.500000</td>
    </tr>
    <tr>
      <th>75%</th>
      <td>4241.500000</td>
      <td>97.000000</td>
      <td>473.000000</td>
    </tr>
    <tr>
      <th>max</th>
      <td>25143.000000</td>
      <td>607.000000</td>
      <td>3951.000000</td>
    </tr>
    <tr>
      <th rowspan="8" valign="top">1</th>
      <th>count</th>
      <td>400.000000</td>
      <td>400.000000</td>
      <td>400.000000</td>
    </tr>
    <tr>
      <th>mean</th>
      <td>6381.175000</td>
      <td>141.017500</td>
      <td>734.167500</td>
    </tr>
    <tr>
      <th>std</th>
      <td>7694.744864</td>
      <td>147.456879</td>
      <td>893.691446</td>
    </tr>
    <tr>
      <th>min</th>
      <td>1128.000000</td>
      <td>26.000000</td>
      <td>123.000000</td>
    </tr>
    <tr>
      <th>25%</th>
      <td>2503.750000</td>
      <td>60.750000</td>
      <td>277.250000</td>
    </tr>
    <tr>
      <th>50%</th>
      <td>3940.000000</td>
      <td>91.500000</td>
      <td>446.500000</td>
    </tr>
    <tr>
      <th>75%</th>
      <td>7081.000000</td>
      <td>156.250000</td>
      <td>788.750000</td>
    </tr>
    <tr>
      <th>max</th>
      <td>59292.000000</td>
      <td>958.000000</td>
      <td>7781.000000</td>
    </tr>
  </tbody>
</table>
</div>

In [35]:
# extracting tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer


def get_lemmas(text):
    words = TextBlob(text).words
    return [word.lemma for word in words]

tf_idf = TfidfVectorizer(analyzer=get_lemmas)

In [36]:
from sklearn.naive_bayes import MultinomialNB
import sklearn.pipeline

mnb = sklearn.pipeline.make_pipeline(
    tf_idf, 
    MultinomialNB()
)

cv = sklearn.cross_validation.StratifiedKFold(
    features.label, n_folds=5, shuffle=True, random_state=30)
scores = sklearn.cross_validation.cross_val_score(
    mnb, features.text, features.label, cv=cv, verbose=1, scoring='accuracy',
)
print(scores, scores.mean())

[ 0.94375  0.96875  0.9375   0.925    0.91875] 0.93875


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.2min finished


In [37]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

svm = Pipeline([
    ('tf_idf', tf_idf),
    ('svm', LinearSVC()),
])

param_grid = [
    {'svm__C': [10, 1000, 1, 1e2, 0.1]},
]

cv = sklearn.cross_validation.StratifiedKFold(
    features.label, n_folds=5, shuffle=True, random_state=30)
grid = GridSearchCV(svm, param_grid,
                    cv=cv, refit=True,
                    scoring='accuracy')
svm_grid = grid.fit(features.text, features.label)
svm_grid.grid_scores_

[mean: 0.98250, std: 0.00612, params: {'svm__C': 10},
 mean: 0.98250, std: 0.00612, params: {'svm__C': 1000},
 mean: 0.97875, std: 0.00500, params: {'svm__C': 1},
 mean: 0.98250, std: 0.00612, params: {'svm__C': 100.0},
 mean: 0.95875, std: 0.01403, params: {'svm__C': 0.1}]

In [43]:
import numpy as np

tfidf = svm_grid.best_estimator_.named_steps['tf_idf']

feature_array = np.array(
    svm_grid.best_estimator_.named_steps['tf_idf'].get_feature_names()
)
f_weights = svm_grid.best_estimator_.named_steps['svm'].coef_[0]
tfidf_sorting = np.argsort(np.abs(f_weights)).flatten()[-30:]
for i in tfidf_sorting:
    print(feature_array[i], f_weights[i])
feature_array[tfidf_sorting]

Perl -0.67938002301
ntserver.yueli.com.tw 0.68254589509
yahoogroups.com -0.696253407767
that -0.697498729825
netnoteinc.com 0.715677069961
lugh.tuatha.org -0.718067907186
virtual.auracom.net 0.72741136084
mandark.labs.netnoteinc.com 0.731178712184
br 0.733158303702
localhost -0.737388136935
0102 0.746504964554
mail -0.769452290101
our 0.792902229701
00 -0.795721580996
000000000000000000000 0.806941663212
mailto -0.817696646261
WARNING -0.827615089404
May 0.855792644263
it -0.862684746414
yahoo.com 0.875844425995
ftp -0.904529673184
FREE 0.935499917947
19 -0.991749729567
petting-zoo.net -1.04627418785
freshrpms.net -1.06802436832
the -1.11483282804
linux.ie -1.16584323234
you 1.28025327553
Aug -1.41404691745
your 1.53949420553


array(['Perl', 'ntserver.yueli.com.tw', 'yahoogroups.com', 'that',
       'netnoteinc.com', 'lugh.tuatha.org', 'virtual.auracom.net',
       'mandark.labs.netnoteinc.com', 'br', 'localhost', '0102', 'mail',
       'our', '00', '000000000000000000000', 'mailto', 'WARNING', 'May',
       'it', 'yahoo.com', 'ftp', 'FREE', '19', 'petting-zoo.net',
       'freshrpms.net', 'the', 'linux.ie', 'you', 'Aug', 'your'], 
      dtype='<U984')

In [51]:
def load_files_with_names(path: str) -> Iterator[Tuple[str, str]]:
    for file in os.listdir(path):
        with open(os.path.join(path, file)) as f:
            yield file, f.read()

unknown_files = pd.DataFrame.from_records(
    load_files_with_names('data/unknown'), columns=('file_name', 'text'))

In [54]:
unknown_files['predictions'] = svm_grid.best_estimator_.predict(unknown_files.text)

In [58]:
unknown_files.to_csv('data/result.csv', columns=["file_name", "predictions"],
                     sep=';', index=None)